## データの読み込み

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 環境の問題：　 libiomp5.dylib というファイルがダブってる。その対応
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [3]:
import tensorflow as tf

In [4]:
import os 
import json

# コーパスのディレクトリを指定
file_path = "/content/drive/MyDrive/rest1046/"
# ファイルの一覧を取得
file_dir = os.listdir(file_path)
# 人間の発話を保持するリスト
utterance_txt = []
# システムの応答を保持するリスト
system_txt = []
# 正解ラベルを保持するリスト
label = []

# ファイルごとに対話データを整形する
for file in file_dir:
    # read json
    r = open(file_path + file, "r", encoding="utf-8")
    json_data = json.load(r)
    
    # 発話データ配列から発話テキストと破綻かどうかの正解データの抽出
    for turn in json_data["turns"]:
        turn_index = turn["turn-index"] # turn-indexキー（対話のインデックス）
        speaker = turn["speaker"] # speakerキー（”U"人間、”S"システム）
        utterance = turn["utterance"] # utteranceキー（発話テキスト）
        
        # 先頭行（システムの冒頭の発話）以外の処理
        if turn_index != 0:
            # 人間の発話テキストを抽出
            if speaker == "U":
                # u_text = ""
                u_text = utterance
            # システムの応答内容が破綻かどうかを抽出
            else:
                a = ""
                sys = turn["utterance"] # システムの発話（応答）を抽出
                t = turn["annotations"][0] # 1つ目のアノテーションを抽出
                a = t["breakdown"] # アノテーションのフラグを抽出
                if a == "O": # O(破綻していない)を0で置換
                    val = 0
                elif a == "T": # T(破綻していないが違和感がある）を1で置換
                    val = 1
                else:
                    val = 2
                # 人間の発話をリストに追加
                utterance_txt.append(u_text)
                # システムの応答をリストに追加
                system_txt.append(sys)
                label.append(str(val))

## 読み込んだデータのサイズを出力

In [5]:
print(len(utterance_txt))
print(len(system_txt))
print(len(label))

10460
10460
10460


## utterance_txtに格納されたテキストを出力する

In [6]:
utterance_txt

['こんにちはー。いい案だね',
 'もう海を諦めるのか',
 '僕には似合わないかな。',
 'しないしない',
 'いや、いいってば。',
 'どういたしまして。',
 '釣り好きなの？',
 '釣りを？',
 '語学と釣りを一緒にやるんだ。',
 'いってらっしゃい。',
 'こんにちはー',
 'ちゃー',
 'っす',
 '挨拶はもういいよ',
 '君もね',
 'なにが？',
 'やるの？',
 '二人だけだとつまらないよ',
 '1対1だよ？',
 'いいんだ…',
 'おはようございます',
 '音楽はお好きですか',
 '音楽はいいですよね',
 'ライブにはあまり行かないです．',
 'まあそういうことはありますね',
 '必要と言うか，ないとさびしいですね',
 '重要です',
 'クラシック音楽が好きなんですよ',
 'メンデルスゾーンとか',
 'ですよね',
 'こんにちは',
 '最近，運動してますか',
 '運動するといいですよ',
 'ウォーキングとか',
 '続けるのは難しいですよね',
 '私は好きですよ',
 'ウォーキングとか登山とか',
 '楽しいですよ．自分のペースでできますしね',
 '気持ちがいいです．ロングウォークが好みです',
 '話しが合いますね',
 'こんにちは',
 '昼から会議なんですよ',
 '臨時の会議です',
 'はっきり言って面倒です',
 '夕方はないです',
 '１３：３０からです',
 'まあそうですね．自分は担当ではないので，今回の会議は特に準備は不要です',
 '担当を決めないと進まないですからね',
 'そうですね．それぞれが何かを担当しています',
 'あれこれと．',
 'そうだね。暑いしね。',
 '暑いから夕方からでいいかな',
 '楽しいよね！',
 'サーフィンしたいな',
 '見ているよりする方が楽しいよね',
 'うまくないけど、好きだよ。',
 'ありがとう！一緒にやる？',
 'きもちが大事。',
 '何事も気持ちから。',
 '元気そうで何よりだよ。',
 '冷やして塩かけると最高ですね。',
 '僕もです。',
 '一人暮らしになってからは、スイカじゃなくてソルダムを良く買うかな。',
 '夏場に良く食べる物って何？',
 'ついてないかぁ。',
 '残念じゃ

## システムの応答を出力

In [7]:
system_txt

['案は凍結が決定されますか？',
 '海が似合ってますか？',
 '海で釣りしますか？',
 'しようか',
 'ありがとー',
 'はい',
 '韓国語で教えるかもしれない',
 '釣りが一緒にできます',
 '早朝から釣りに行きます',
 'いってらっしゃいありがとうございます',
 'こんちゃー',
 'ちゃっす',
 'っす',
 '元気がいいですね',
 'なに',
 'サッカー',
 'いいよ',
 'サッカーは大丈夫です',
 '1対1が良いですよねえ',
 '1対1はいいですよねえ',
 'はいおはよー',
 '心から音楽を楽しんでるんです',
 '音楽はライブがいいですね',
 '音楽は生がいいですね',
 '音楽は必要か？？',
 '音楽は重要ですね',
 '音楽はかっこよいですね',
 '音楽はかわいいですね',
 'メンデルスゾーンの音楽は素晴らしいと思う。',
 'うん',
 'ごきげんよう',
 '退屈は嫌です',
 '退屈を嫌います',
 'ウォーキングをサボります',
 'ウォーキングを楽しむんでしょう？',
 'ありがとう',
 'ウォーキングは楽しいですね',
 '気持ちがいいですね',
 'ウォーキングはいいですね',
 '話しは好きですね',
 'こんちは',
 '会議が急に入りますか？？',
 '会議に入ってますか？',
 '会議が夕方に入ってます？',
 '会議が時間に入ってます？',
 '準備が大切ですね',
 '担当は必要ですか',
 'それぞれが担当を持ちますか',
 '担当が言います？？',
 '担当はいいですね',
 '朝から海に行きますか？？',
 '海は楽しいですね',
 '楽しいよー',
 'サーフィンはするのがいいですね',
 'サーフィンがうまいですね',
 'サーフィンは上手いですね',
 'サーフィンはスタイルが大事か？？',
 'きもちはうれしいですね',
 '私も大好きです！',
 '私も大好きです！',
 '大好きです',
 'ねー',
 'ついていないのが残念です',
 'ついていないのが残念よねえ',
 'ついていないのが残念じゃないですよ',
 'そうですそうです',
 'うん笑',
 'なんだよ',
 'なんだよ',
 'なんだよ',
 'スイカで買い物できてるのですか？',
 '買

## 正解ラベルを出力

In [8]:
label

['2',
 '1',
 '0',
 '1',
 '2',
 '2',
 '2',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '2',
 '2',
 '1',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '2',
 '0',
 '1',
 '2',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '2',
 '1',
 '0',
 '0',
 '0',
 '2',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '2',
 '0',
 '2',
 '1',
 '2',
 '0',
 '0',
 '1',
 '2',
 '2',
 '0',
 '0',
 '1',
 '2',
 '2',
 '2',
 '0',
 '1',
 '0',
 '0',
 '1',
 '2',
 '2',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '2',
 '1',
 '0',
 '2',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '2',
 '2',
 '2',
 '1',
 '0',
 '0',
 '1',
 '2',
 '0',
 '1',
 '1',
 '2',
 '0',
 '2',
 '0',
 '0',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '0',
 '0',
 '0',
 '2',
 '1',
 '0',
 '0',
 '2',
 '1',
 '0',
 '1',
 '1',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1'

## データフレームにまとめる

In [9]:
import pandas as pd
df = pd.DataFrame({"utterance_txt": utterance_txt,
                                  "system_txt": system_txt,
                                  "label": label}
                                 )
df

,utterance_txt,system_txt,label
0,こんにちはー。いい案だね,案は凍結が決定されますか？,2
1,もう海を諦めるのか,海が似合ってますか？,1
2,僕には似合わないかな。,海で釣りしますか？,0
3,しないしない,しようか,1
4,いや、いいってば。,ありがとー,2
...,...,...,...
10455,もともとはスコットランド民謡なんですよね。,まだまだ、いけます。,2
10456,良い曲は何年前の曲でもやっぱりいいです。,いいなあ！,1
10457,むしろ最近の曲は全然聞かないですね…,いいなあ！,2
10458,いやいや、全然よくないよ。良い曲いっぱい出てきてほしい。,楽しみです,0


## カウントする

In [10]:
df["label"].value_counts()

0    5329
1    2937
2    2194
Name: label, dtype: int64

## 形態素への分解

In [11]:
!pip install janome

     |████████████████████████████████| 19.7 MB 7.7 MB/s 


In [12]:
from janome.tokenizer import Tokenizer
import re

def parse(utterance_txt):
    t = Tokenizer() # Tokenizerクラスのオブジェクトを生成
    separation_tmp = [] # 形態素を一時保存するリスト
    # 形態素に分解
    for row in utterance_txt:
        # リストから発話テキストの部分を抽出して形態素解析を実行
        tokens = t.tokenize(row)
        # 形態素の見出しの部分を取得してseparation_tmpに追加
        separation_tmp.append(
                [token.surface for token in tokens if(
                        not re.match("記号", token.part_of_speech) # 記号を除外
                        and (not re.match("助詞", token.part_of_speech)) # 助詞を除外
                        and (not re.match("助動詞", token.part_of_speech)) # 助動詞を除外
                        )
                ])
        # 空の要素があれば取り除く
        while separation_tmp.count("") > 0:
            separation_tmp.remove("")
    return separation_tmp
    
# 人間の発話を形態素に分解する
df["utterance_token"] = parse(df["utterance_txt"])
# システムの応答を形態素に分解する
df["system_token"] = parse(df["system_txt"])
# 形態素への分解後のデータフレームを出力
df

,utterance_txt,system_txt,label,utterance_token,system_token
0,こんにちはー。いい案だね,案は凍結が決定されますか？,2,"[こんにちは, ー, いい, 案]","[案, 凍結, 決定, さ, れ]"
1,もう海を諦めるのか,海が似合ってますか？,1,"[もう, 海, 諦める, の]","[海, 似合っ]"
2,僕には似合わないかな。,海で釣りしますか？,0,"[僕, 似合わ]","[海, 釣り, し]"
3,しないしない,しようか,1,"[し, し]",[しよ]
4,いや、いいってば。,ありがとー,2,"[いや, いい, ば]","[ありがと, ー]"
...,...,...,...,...,...
10455,もともとはスコットランド民謡なんですよね。,まだまだ、いけます。,2,"[もともと, スコットランド, 民謡, ん]","[まだまだ, いけ]"
10456,良い曲は何年前の曲でもやっぱりいいです。,いいなあ！,1,"[良い, 曲, 何, 年, 前, 曲, やっぱり, いい]",[いい]
10457,むしろ最近の曲は全然聞かないですね…,いいなあ！,2,"[むしろ, 最近, 曲, 全然, 聞か]",[いい]
10458,いやいや、全然よくないよ。良い曲いっぱい出てきてほしい。,楽しみです,0,"[いやいや, 全然, よく, 良い, 曲, いっぱい, 出, き, ほしい]",[楽しみ]


## 形態素の数をデータフレームに登録

In [13]:
df["u_token_len"] = df["utterance_token"].apply(len)
df["s_token_len"] = df["system_token"].apply(len)
df

,utterance_txt,system_txt,label,utterance_token,system_token,u_token_len,s_token_len
0,こんにちはー。いい案だね,案は凍結が決定されますか？,2,"[こんにちは, ー, いい, 案]","[案, 凍結, 決定, さ, れ]",4,5
1,もう海を諦めるのか,海が似合ってますか？,1,"[もう, 海, 諦める, の]","[海, 似合っ]",4,2
2,僕には似合わないかな。,海で釣りしますか？,0,"[僕, 似合わ]","[海, 釣り, し]",2,3
3,しないしない,しようか,1,"[し, し]",[しよ],2,1
4,いや、いいってば。,ありがとー,2,"[いや, いい, ば]","[ありがと, ー]",3,2
...,...,...,...,...,...,...,...
10455,もともとはスコットランド民謡なんですよね。,まだまだ、いけます。,2,"[もともと, スコットランド, 民謡, ん]","[まだまだ, いけ]",4,2
10456,良い曲は何年前の曲でもやっぱりいいです。,いいなあ！,1,"[良い, 曲, 何, 年, 前, 曲, やっぱり, いい]",[いい],8,1
10457,むしろ最近の曲は全然聞かないですね…,いいなあ！,2,"[むしろ, 最近, 曲, 全然, 聞か]",[いい],5,1
10458,いやいや、全然よくないよ。良い曲いっぱい出てきてほしい。,楽しみです,0,"[いやいや, 全然, よく, 良い, 曲, いっぱい, 出, き, ほしい]",[楽しみ],9,1


## 単語の出現回数を記録して辞書を作成

In [14]:
from collections import Counter # カウント処理のためのライブラリ
import itertools # イテレーションのためのライブラリ

# {単語：出現回数}の辞書を作成
def makedictionary(data):
    return Counter(itertools.chain(* data))

## 単語を出現回数順（降順）に並び替えて連番をふる関数

In [15]:
def update_word_dictionary(worddic):
    word_list = []
    word_dic = {}
    # most_common()で出現回数順に要素を取得しword_listに追加
    for w in worddic.most_common():
        word_list.append(w[0])
    
    # 出現回数順に並べた単語をキーに、1から始まる連番を値に設定
    for i, word in enumerate(word_list, start=1):
        word_dic.update({word : i})
    
    return word_dic

## 単語を出現回数の数値に置き換える関数の定義

In [16]:
def bagOfWords(word_dic, token):
    return [[word_dic[word] for word in sp] for sp in token]

## 発話を{単語：出現回数の辞書にする}


In [17]:
utter_word_frequency = makedictionary(df["utterance_token"])
utter_word_frequency

Counter({'こんにちは': 187,
         'ー': 136,
         'いい': 566,
         '案': 3,
         'もう': 73,
         '海': 235,
         '諦める': 1,
         'の': 598,
         '僕': 120,
         '似合わ': 3,
         'し': 739,
         'いや': 88,
         'ば': 2,
         'どういたしまして': 29,
         '釣り': 19,
         '好き': 665,
         '語学': 2,
         '一緒': 68,
         'やる': 15,
         'ん': 777,
         'いっ': 16,
         'らっしゃい': 5,
         'ちゃ': 1,
         '挨拶': 1,
         '君': 46,
         'なに': 46,
         '二': 12,
         '人': 153,
         'つまらない': 4,
         '1': 17,
         '対': 1,
         'おはよう': 40,
         '音楽': 32,
         'お': 197,
         'ライブ': 5,
         'あまり': 62,
         '行か': 43,
         'まあ': 60,
         'そういう': 37,
         'こと': 402,
         'あり': 336,
         '必要': 41,
         '言う': 14,
         'ない': 152,
         'さびしい': 3,
         '重要': 12,
         'クラシック': 2,
         'メンデルスゾーン': 1,
         '最近': 170,
         '運動': 20,
         'する': 172,
         

## 応答を{単語：出現回数}の辞書にする

In [18]:
system_word_frequency = makedictionary(df["system_token"])
system_word_frequency

Counter({'案': 2,
         '凍結': 1,
         '決定': 1,
         'さ': 101,
         'れ': 90,
         '海': 386,
         '似合っ': 1,
         '釣り': 12,
         'し': 171,
         'しよ': 2,
         'ありがと': 38,
         'ー': 153,
         'はい': 93,
         '韓国': 27,
         '語': 31,
         '教える': 14,
         'しれ': 104,
         '一緒': 114,
         'でき': 59,
         '早朝': 1,
         '行き': 167,
         'いっ': 8,
         'らっしゃい': 5,
         'ありがとう': 107,
         'こん': 17,
         'ちゃ': 1,
         '元気': 32,
         'いい': 847,
         'なに': 33,
         'サッカー': 32,
         '大丈夫': 119,
         '1': 7,
         '対': 2,
         '良い': 162,
         'お': 130,
         '心から': 3,
         '音楽': 48,
         '楽しん': 27,
         'でる': 39,
         'ん': 432,
         'ライブ': 6,
         '生': 16,
         '必要': 48,
         '重要': 20,
         'かっこよい': 6,
         'かわいい': 15,
         'メンデルスゾーン': 1,
         '素晴らしい': 23,
         '思う': 3,
         'うん': 161,
         'ごきげんよう': 4,
         '退屈

## 発話の単語辞書を単語の出現回数順（降順）で並び替えて連番を割り当てる

In [19]:
utter_word_dic = update_word_dictionary(utter_word_frequency)
utter_word_dic

{'ん': 1,
 'し': 2,
 'そう': 3,
 '好き': 4,
 'の': 5,
 'いい': 6,
 '何': 7,
 'こと': 8,
 '私': 9,
 'い': 10,
 'あり': 11,
 'そうですね': 12,
 '行き': 13,
 '食べ': 14,
 '海': 15,
 'スイカ': 16,
 'てる': 17,
 'それ': 18,
 'お': 19,
 'はい': 20,
 'こんにちは': 21,
 'ある': 22,
 '気': 23,
 'する': 24,
 'いる': 25,
 '最近': 26,
 'あなた': 27,
 'よく': 28,
 'どこ': 29,
 'どう': 30,
 '見': 31,
 '夏': 32,
 'どんな': 33,
 '人': 34,
 'ない': 35,
 '今日': 36,
 '思い': 37,
 '暑い': 38,
 'ー': 39,
 'もの': 40,
 '行っ': 41,
 '良い': 42,
 '僕': 43,
 '楽しい': 44,
 'て': 45,
 'なり': 46,
 'おいしい': 47,
 '方': 48,
 '退屈': 49,
 'でも': 50,
 'なっ': 51,
 '行く': 52,
 '今': 53,
 'ほう': 54,
 'つけ': 55,
 'いや': 56,
 '大丈夫': 57,
 'ありがとう': 58,
 'さ': 59,
 'うん': 60,
 '多い': 61,
 'ください': 62,
 'れ': 63,
 '仕事': 64,
 '熱中': 65,
 '症': 66,
 '大変': 67,
 '美味しい': 68,
 'き': 69,
 'なる': 70,
 'こんばんは': 71,
 'もう': 72,
 'でき': 73,
 'よう': 74,
 'いえ': 75,
 '雨': 76,
 'やっ': 77,
 '一緒': 78,
 'とき': 79,
 '話': 80,
 '嫌い': 81,
 '確か': 82,
 'みたい': 83,
 'ゲーム': 84,
 '山': 85,
 'ちょっと': 86,
 'あまり': 87,
 '見る': 88,
 '他': 89,
 'まあ': 90,
 'しれ': 91,
 '今年'

## 応答の単語辞書を単語の出現回数順（降順）で並び替えて連番を割り当てる

In [20]:
system_word_dic = update_word_dictionary(system_word_frequency)
system_word_dic

{'いい': 1,
 'の': 2,
 '好き': 3,
 'ん': 4,
 '海': 5,
 'そう': 6,
 'スイカ': 7,
 '退屈': 8,
 '症': 9,
 '熱中': 10,
 'てる': 11,
 '大好き': 12,
 'し': 13,
 '行き': 14,
 '気': 15,
 '良い': 16,
 'うん': 17,
 '多い': 18,
 'ー': 19,
 'つけ': 20,
 '楽しい': 21,
 'こと': 22,
 '欲しい': 23,
 'お': 24,
 'よう': 25,
 '心': 26,
 '雨': 27,
 '美味しい': 28,
 '人': 29,
 '大丈夫': 30,
 '一緒': 31,
 '気持ち': 32,
 'ありがとう': 33,
 '有名': 34,
 'あり': 35,
 'しれ': 36,
 'て': 37,
 '見': 38,
 'さ': 39,
 '哲学': 40,
 'い': 41,
 'はい': 42,
 'れ': 43,
 '何': 44,
 'ない': 45,
 '大切': 46,
 '面白い': 47,
 '予防': 48,
 '朝': 49,
 'する': 50,
 '趣味': 51,
 '行く': 52,
 '映画': 53,
 '高い': 54,
 '仕事': 55,
 'うまい': 56,
 '食べ': 57,
 '楽しみ': 58,
 '買い物': 59,
 '旅行': 60,
 '実': 61,
 '一': 62,
 '天気': 63,
 '行っ': 64,
 '深い': 65,
 '食べる': 66,
 'でき': 67,
 'ゲーム': 68,
 'なり': 69,
 '見る': 70,
 '行か': 71,
 '大事': 72,
 '花火': 73,
 '音楽': 74,
 '必要': 75,
 '奥': 76,
 'テレビ': 77,
 'とり': 78,
 '大きい': 79,
 '強い': 80,
 '悪い': 81,
 'プール': 82,
 '味': 83,
 'スポーツ': 84,
 '予定': 85,
 'しり': 86,
 '綺麗': 87,
 'でる': 88,
 '気分': 89,
 'ありがと': 90,
 'どう': 91,
 'クラゲ'

## 辞書のサイズを変数に登録する

In [21]:
utter_dic_size = len(utter_word_dic)
system_dic_size = len(system_word_dic)
print(utter_dic_size)
print(system_dic_size)

5330
4428


## 単語を出現回数順の数値に置き換える

In [22]:
train_utter = bagOfWords(utter_word_dic, df["utterance_token"])
train_system = bagOfWords(system_word_dic, df["system_token"])

## 数値に置き換えた発話を出力する

In [23]:
train_utter

[[21, 39, 6, 1270],
 [72, 15, 2624, 5],
 [43, 1271],
 [2, 2],
 [56, 6, 1737],
 [195],
 [286, 4],
 [286],
 [1738, 286, 78, 354, 1],
 [335, 868],
 [21, 39],
 [2625, 39],
 [],
 [2626, 72, 6],
 [114],
 [115],
 [354],
 [433, 34, 1034],
 [323, 2627, 323],
 [6, 1],
 [135],
 [166, 19, 4],
 [166, 6],
 [869, 87, 126],
 [90, 145, 8, 11],
 [131, 374, 35, 1272],
 [434],
 [1739, 166, 4, 1],
 [2628],
 [],
 [21],
 [26, 273, 2],
 [273, 24, 6],
 [1740],
 [1273, 5, 150],
 [9, 4],
 [1740, 2629],
 [44, 177, 2630, 73],
 [232, 6, 2631, 2632, 661],
 [196, 601],
 [21],
 [336, 662, 1],
 [2633, 662],
 [1274, 186, 557],
 [740],
 [602, 1275, 1275, 1741],
 [90, 3, 177, 1276, 1277, 662, 112, 663, 2634],
 [1276, 1035, 1742],
 [12, 1278, 7, 1276, 2, 10],
 [2635],
 [3, 38],
 [38, 740, 6],
 [44],
 [558, 2],
 [31, 25, 24, 48, 44],
 [515, 4],
 [58, 78, 354],
 [69, 1743, 93],
 [870, 232],
 [156, 3, 2636],
 [516, 146, 337, 197],
 [43],
 [603, 51, 16, 1744, 1279, 140, 471],
 [741, 140, 102, 405, 7],
 [1745],
 [375],
 [3, 1],

## 数値に置き換えた応答を出力する

In [24]:
train_system

[[1738, 2582, 2583, 39, 43],
 [5, 2584],
 [5, 354, 13],
 [1739],
 [90, 19],
 [42],
 [137, 113, 300, 36],
 [354, 31, 67],
 [2585, 354, 14],
 [504, 807, 33],
 [240, 19],
 [2586],
 [],
 [107, 1],
 [102],
 [108],
 [1],
 [108, 30],
 [575, 1740, 575, 16],
 [575, 1740, 575, 1],
 [42, 24],
 [1287, 74, 138, 88, 4],
 [74, 681, 1],
 [74, 255, 1],
 [74, 75],
 [74, 198],
 [74, 682],
 [74, 274],
 [2587, 74, 168, 1288],
 [17],
 [1008],
 [8, 97],
 [8, 128],
 [1009, 1741],
 [1009, 157, 4],
 [33],
 [1009, 21],
 [32, 1],
 [1009, 1],
 [505, 3],
 [456],
 [576, 683, 169],
 [576, 388],
 [576, 808, 388],
 [576, 158, 388],
 [1289, 46],
 [1010, 75],
 [2588, 1010, 241],
 [1010, 684],
 [1010, 1],
 [49, 5, 14],
 [5, 21],
 [21],
 [170, 50, 2, 1],
 [170, 56],
 [170, 242],
 [170, 809, 72],
 [577, 2589, 1011],
 [184, 12],
 [184, 12],
 [12],
 [],
 [355, 41, 2, 457],
 [355, 41, 2, 457],
 [355, 41, 2, 457],
 [6, 6],
 [17, 209],
 [144],
 [144],
 [144],
 [7, 59, 67, 11, 2],
 [59, 243, 506, 1],
 [321, 14],
 [31],
 [6],
 [68

## 発話と応答それぞれの形態素の最大数を取得する

In [25]:
UTTER_MAX_SIZE = len(sorted(train_utter, key=len, reverse=True)[0])
SYSTEM_MAX_SIZE = len(sorted(train_system, key=len, reverse=True)[0])
print(UTTER_MAX_SIZE)
print(SYSTEM_MAX_SIZE)

18
41


## 単語データの配列を同一のサイズに揃える関数を定義する

In [26]:
from tensorflow.keras.preprocessing import sequence

def padding_sequences(data, max_len): # 最長のサイズになるまでゼロを埋め込む
    return sequence.pad_sequences(
            data, maxlen=max_len, padding="post", value=0.0)


## 発話の単語配列のサイズを最長サイズに合わせる

In [27]:
train_U = padding_sequences(train_utter, UTTER_MAX_SIZE)
print(train_U.shape)
print(train_U)

(10460, 18)
[[  21   39    6 ...    0    0    0]
 [  72   15 2624 ...    0    0    0]
 [  43 1271    0 ...    0    0    0]
 ...
 [1160   26  326 ...    0    0    0]
 [ 458  494   28 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]


## 応答の単語配列のサイズを最長サイズに合わせる

In [28]:
train_S = padding_sequences(train_system, SYSTEM_MAX_SIZE)
print(train_S.shape)
print(train_S)

(10460, 41)
[[1738 2582 2583 ...    0    0    0]
 [   5 2584    0 ...    0    0    0]
 [   5  354   13 ...    0    0    0]
 ...
 [   1    0    0 ...    0    0    0]
 [  58    0    0 ...    0    0    0]
 [1485    1    0 ...    0    0    0]]


## RNNモデルの構築

In [29]:
'''
25. RNNモデルの構築
'''
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, GRU, Embedding, Flatten, Dropout
from tensorflow.keras import models, layers, optimizers, regularizers

## ------入力層------
# 人間の発話:ユニット数は単語配列の最長サイズと同じ
utterance = Input(shape=(UTTER_MAX_SIZE,), name='utterance')
# システムの応答:ユニット数は単語配列の最長サイズと同じ
system = Input(shape=(SYSTEM_MAX_SIZE,), name='system')
# 人間の発話の単語数:ユニット数は1
u_token_len = Input(shape=[1], name="u_token_len")
# システムの応答の単語数:ユニット数は1
s_token_len = Input(shape=[1], name="s_token_len")

# ------Embedding層------
# 人間の発話: 入力は単語の総数+100、出力の次元数は128
emb_utterance = Embedding(
    input_dim=utter_dic_size+100,  # 発話の単語数+100
    output_dim=128,                # 出力の次元数はRecurrent層のユニット数
    )(utterance)
# システムの応答: 入力は単語の総数+100、出力の次元数は128
emb_system = Embedding(
    input_dim=system_dic_size+100, # 応答の単語数+100
    output_dim=128                  # 出力の次元数はRecuurrent層のユニット数
    )(system)
# 人間の発話の単語数のEmbedding
emb_u_len = Embedding(             # 入力の次元は発話の形態素数の最大値+1
    input_dim=UTTER_MAX_SIZE+1,    # 出力は5
    output_dim=5
    )(u_token_len)
# システムの応答の単語数のEmbedding
emb_s_len = Embedding(
    input_dim=SYSTEM_MAX_SIZE+1,   # 入力の次元は応答の形態素数の最大値+1
    output_dim=5                   # 出力は5
    )(s_token_len)

# ------Recurrent層------
# 人間の発話: GRUユニット×128×3段
rnn_layer1_1 = GRU(128, return_sequences=True
                  )(emb_utterance)
rnn_layer1_2 = GRU(128, return_sequences=True
                  )(rnn_layer1_1)
rnn_layer1_3 = GRU(128, return_sequences=False
                  )(rnn_layer1_2)

# システムの応答: GRUユニット×128×3段
rnn_layer2_1 = GRU(128, return_sequences=True
                  )(emb_system)
rnn_layer2_2 = GRU(128, return_sequences=True
                  )(rnn_layer2_1)
rnn_layer2_3 = GRU(128, return_sequences=False
                  )(rnn_layer2_2)

# ------全結合層------
main_l = concatenate([
    Flatten()(emb_u_len), # 人間の発話の単語数のEmbedding
    Flatten()(emb_s_len), # 商品名の単語数のEmbedding
    rnn_layer1_3,         # 人間の発話のGRUユニット
    rnn_layer2_3          # システム応答のGRUユニット
])

# ------512、256、128ユニットの層を追加------
main_l = Dropout(0.2)(
    Dense(512,kernel_initializer='normal',activation='relu')(main_l))
main_l = Dropout(0.2)(
    Dense(256,kernel_initializer='normal',activation='relu')(main_l))
main_l = Dropout(0.2)(
    Dense(128,kernel_initializer='normal',activation='relu')(main_l))

# ------出力層(1ユニット)------
output = Dense(units=3,              # 出力層のニューロン数＝2
                activation='softmax' # 活性化はソフトマックス関数
               )(main_l)


# Modelオブジェクトの生成
model = models.Model(
    # 入力層はマルチ入力モデルなのでリストにする
    inputs=[utterance, system,
            u_token_len, s_token_len
           ],
    # 出力層
    outputs=output
)

# Sequentialオブジェクをコンパイル
model.compile(
    loss='categorical_crossentropy', # 誤差関数はクロスエントロピー
    optimizer=optimizers.Adam(),     # Adamオプティマイザー
    metrics=['accuracy']             # 学習評価として正解率を指定
    )

model.summary()                      # RNNのサマリー（概要）を出力

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
utterance (InputLayer)          [(None, 18)]         0                                            
__________________________________________________________________________________________________
system (InputLayer)             [(None, 41)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 18, 128)      695040      utterance[0][0]                  
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 41, 128)      579584      system[0][0]                     
______________________________________________________________________________________________

## 訓練データと正解ラベルの用意

In [30]:
import numpy as np
from tensorflow.keras.utils import to_categorical

trainX = {
    "utterance": train_U,
    "system": train_S,
    "u_token_len": np.array(df[["u_token_len"]]),
    "s_token_len": np.array(df[["s_token_len"]])
}

#正解ラベルをOne -Hot表現にする
trainY = to_categorical(df["label"], 3)


## 学習を実行する

In [31]:
%%time
import math
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping

training_epochs = 120
batch_size = 32
lr_min = 0.0001
lr_max = 0.001

def step_decay(epoch):
    initial_lrate = 0.001 # 学習率の初期値
    drop = 0.5 # 減衰率は25%
    epochs_drop = 10.0 # 10エポックごとに減衰する
    lrate = initial_lrate * math.pow(
            drop,
            math.floor((epoch)/epochs_drop)
    )
    return lrate

lrate = LearningRateScheduler(step_decay)

earlystopping = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5,
                                                 mode='max', verbose=1)
callbacks_list = [lrate, earlystopping]

epoch = 100

# 学習を行う
history = model.fit(trainX, trainY,
        batch_size=batch_size,
        epochs=epoch,
        verbose=1,
        validation_split=0.2,
        shuffle=True,
        callbacks=callbacks_list
)


Epoch 1/100
262/262 [==============================] - 21s 33ms/step - loss: 1.0300 - accuracy: 0.5110 - val_loss: 1.0300 - val_accuracy: 0.5005
Epoch 2/100
262/262 [==============================] - 7s 25ms/step - loss: 1.0232 - accuracy: 0.5117 - val_loss: 1.0294 - val_accuracy: 0.5005
Epoch 3/100
262/262 [==============================] - 7s 25ms/step - loss: 1.0216 - accuracy: 0.5115 - val_loss: 1.0305 - val_accuracy: 0.5005
Epoch 4/100
262/262 [==============================] - 7s 25ms/step - loss: 0.9937 - accuracy: 0.5140 - val_loss: 1.0457 - val_accuracy: 0.5029
Epoch 5/100
262/262 [==============================] - 7s 26ms/step - loss: 0.9130 - accuracy: 0.5692 - val_loss: 1.0730 - val_accuracy: 0.4685
Epoch 6/100
262/262 [==============================] - 7s 25ms/step - loss: 0.8367 - accuracy: 0.6042 - val_loss: 1.0742 - val_accuracy: 0.4484
Epoch 7/100
262/262 [==============================] - 7s 25ms/step - loss: 0.7665 - accuracy: 0.6362 - val_loss: 1.2907 - val_accuracy